In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import glob
from pylab import *
rc('axes',linewidth=2)
alpha=0.7
import seaborn as sns

In [ ]:
read_path = '/path/to/CPM_outputs/output_folder'
json_path = '/path/to/CPM_json/settings.json'

In [ ]:
with open(json_path) as json_data:
    data = json.load(json_data)

In [ ]:
t = data['t']
k = data['k']
p_thresh = data['p_thresh']
repeat = data['repeat']
num_iter = data['num_iter']
mat_path = data['mat_path']
mat_name = data['mat_name']  # should contains behav_name, num_rois, and num_contrasts
zscore = data['zscore']
mode = data['mode']
y_norm = data['y_norm']

In [ ]:
data

# Positive edges

In [ ]:
with open('/Users/fengdanye/Documents/Yale_Research/ABCD/ABCDStudyNDA_Release_3.0/outputs/lst_rois_APARC_ASEG_86ROIs.txt','r') as f:
    lst_rois=f.read().splitlines()

"""
with open('/Users/fengdanye/Documents/Yale_Research/Cannabis_project/lst_nodes_orig.txt','r') as f:
    lst_rois=f.read().splitlines()
"""

In [ ]:
num_nodes=len(lst_rois)

In [ ]:
num_nodes

In [ ]:
thresh = 1.0

In [ ]:
data=np.loadtxt('{}/network_pos_consensus.txt'.format(read_path))
data.shape

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
img = ax.imshow(data, interpolation='nearest', cmap='Reds', alpha=alpha, vmin=0, vmax=1) 
plt.show()

In [ ]:
data_thresh=(data>=thresh).astype(int)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
img = ax.imshow(data_thresh, interpolation='nearest', cmap='Reds', alpha=alpha, vmin=0, vmax=1) 
plt.show()

In [ ]:
# save to file
np.savetxt('{}/net_pos_thresh_{}.txt'.format(read_path, thresh), data_thresh, fmt='%f')

In [ ]:
iu = np.triu_indices(num_nodes, k=1)
lst_sig_edges = []
j=0
for x,y in zip(iu[0],iu[1]):
    j=j+1
    if data_thresh[x,y] == 1:
        lst_sig_edges.append((lst_rois[x],lst_rois[y]))
        

In [ ]:
print(j)
print(len(lst_sig_edges))

In [ ]:
np.sum(data_thresh)/2

In [ ]:
with open('{}/sig_edges_pos_thresh{}.txt'.format(read_path,thresh),'w') as f:
    for (a,b) in lst_sig_edges:
        f.write('{}\t{}\n'.format(a,b))

# Negative edges

In [ ]:
thresh = 1.0

In [ ]:
data=np.loadtxt('{}/network_neg_consensus.txt'.format(read_path))
data.shape

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
img = ax.imshow(data, interpolation='nearest', cmap='Blues', alpha=alpha, vmin=0, vmax=1) 
plt.show()

In [ ]:
data_thresh=(data>=thresh).astype(int)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
img = ax.imshow(data_thresh, interpolation='nearest', cmap='Blues', alpha=alpha, vmin=0, vmax=1) 
plt.show()

In [ ]:
# save to file
np.savetxt('{}/net_neg_thresh_{}.txt'.format(read_path,thresh), data_thresh, fmt='%f')

In [ ]:
iu = np.triu_indices(num_nodes, k=1)
lst_sig_edges = []
j=0
for x,y in zip(iu[0],iu[1]):
    j=j+1
    if data_thresh[x,y] == 1:
        lst_sig_edges.append((lst_rois[x],lst_rois[y]))
        

In [ ]:
print(j)
print(len(lst_sig_edges))

In [ ]:
np.sum(data_thresh)/2

In [ ]:
with open('{}/sig_edges_neg_thresh{}.txt'.format(read_path,thresh),'w') as f:
    for (a,b) in lst_sig_edges:
        f.write('{}\t{}\n'.format(a,b))

# Rearrange matrix based on canonical networks
Run organize_node_net.m and rearr_canonical_networks.m first.

In [ ]:
thresh=1.0

In [ ]:
module_names = [f for f in glob.glob('{}/canonical_networks/module*_comp_realvalued.txt'.format(read_path)) if "modules_comp" not in f]
module_names.sort(key=lambda f: int(''.join(filter(str.isdigit, f.split('/')[-1]))))

In [ ]:
module_names

In [ ]:
print(len(module_names))
module_size = []
for fn in module_names:
    with open(fn, 'r') as f:
        lst = f.read().splitlines()
        module_size.append(len(lst))

print(len(module_size))
print(sum(module_size))

In [ ]:
module_size

In [ ]:
module_colors=['#2f4f4f', '#228b22', '#7f0000','#ffff00','#0000cd','#00ff00','#00ffff','#1e90ff','#ffe4b5']
print(len(module_colors))

In [ ]:
with open('{}/canonical_networks/modules_comp_realvalued.txt'.format(read_path), 'r') as f:
    lst_rois=f.read().splitlines()

print(len(lst_rois))

In [ ]:
lst_nets = ['cerebellum', 'cingulate', 'frontal', 'insula', 'limbic', 'occipital', 'parietal', 'striatum', 'temporal']
print(len(lst_nets))

#### Positive

In [ ]:
data = np.loadtxt('{}/canonical_networks/correlation_canonical_rearr_pos.txt'.format(read_path), delimiter=',')
np.shape(data)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
img = ax.imshow(data, interpolation='nearest', cmap='Reds', alpha=alpha, vmin=0, vmax=1) 
plt.show()

In [ ]:
data_thresh=(data>=thresh).astype(int)
fig, ax = plt.subplots(figsize=(5,5))
img = ax.imshow(data_thresh, interpolation='nearest', cmap='Reds', alpha=alpha, vmin=0, vmax=1) 
plt.show()

In [ ]:
df=pd.DataFrame(index=lst_nets,columns=lst_nets)
df_ratio=pd.DataFrame(index=lst_nets,columns=lst_nets)
df

In [ ]:
for i in range(9):
    s1=module_size[i]
    l1=np.sum(module_size[:i]).astype(int)
    h1=(l1+s1).astype(int)
    net1=lst_nets[i]
    for j in range(9):
        s2=module_size[j]
        l2=np.sum(module_size[:j]).astype(int)
        h2=(l2+s2).astype(int)
        net2=lst_nets[j]
        print("({},{}): {}, {}. l1 {} - h1 {}. l2 {} - h2 {}.".format(i,j,net1,net2,l1,h1,l2,h2))
        num_edges = np.sum(data_thresh[l1:h1,l2:h2])
        if i==j:
            num_edges = num_edges/2
            network_size = s1*(s1-1)/2
        else:
            network_size = s1*s2
        df.loc[net1, net2]=num_edges
        df_ratio.loc[net1,net2]=num_edges/float(network_size)

In [ ]:
df=df.astype(float)
df_ratio=df_ratio.astype(float)
df

In [ ]:
df_ratio

In [ ]:
mask = np.triu(np.ones_like(df, dtype=bool),k=1)
f, ax = plt.subplots(figsize=(6,6))
hmp=sns.heatmap(df, mask=mask, cmap='Reds',annot=True,linewidths=.5,square=True,linecolor='white',cbar_kws={"shrink": 0.7},annot_kws={"fontsize":12},fmt='g')
ax.set_yticklabels(lst_nets,fontsize=12)
ax.set_xticklabels(lst_nets,fontsize=12)
plt.savefig("{}/canonical_networks/network_pos_canonical_grid_plot_thresh{}.png".format(read_path,thresh), format='png',bbox_inches="tight",dpi=300)

In [ ]:
mask = np.triu(np.ones_like(df_ratio, dtype=bool),k=1)
f, ax = plt.subplots(figsize=(6,6))
hmp=sns.heatmap(df_ratio, mask=mask, cmap='Reds',annot=True,linewidths=.5,square=True,linecolor='white',cbar_kws={"shrink": 0.7},annot_kws={"fontsize":10},fmt='.2f')
ax.set_yticklabels(lst_nets,fontsize=12)
ax.set_xticklabels(lst_nets,fontsize=12)
plt.savefig("{}/canonical_networks/network_pos_canonical_grid_plot_thresh{}_ratio.png".format(read_path,thresh), format='png',bbox_inches="tight",dpi=300)

#### Negative

In [ ]:
data = np.loadtxt('{}/canonical_networks/correlation_canonical_rearr_neg.txt'.format(read_path), delimiter=',')
np.shape(data)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
img = ax.imshow(data, interpolation='nearest', cmap='Blues', alpha=alpha, vmin=0, vmax=1) 
plt.show()

In [ ]:
data_thresh=(data>=thresh).astype(int)
fig, ax = plt.subplots(figsize=(5,5))
img = ax.imshow(data_thresh, interpolation='nearest', cmap='Blues', alpha=alpha, vmin=0, vmax=1) 
plt.show()

In [ ]:
df=pd.DataFrame(index=lst_nets,columns=lst_nets)
df_ratio=pd.DataFrame(index=lst_nets,columns=lst_nets)
df

In [ ]:
for i in range(9):
    s1=module_size[i]
    l1=np.sum(module_size[:i]).astype(int)
    h1=(l1+s1).astype(int)
    net1=lst_nets[i]
    for j in range(9):
        s2=module_size[j]
        l2=np.sum(module_size[:j]).astype(int)
        h2=(l2+s2).astype(int)
        net2=lst_nets[j]
        print("({},{}): {}, {}. l1 {} - h1 {}. l2 {} - h2 {}.".format(i,j,net1,net2,l1,h1,l2,h2))
        num_edges = np.sum(data_thresh[l1:h1,l2:h2])
        if i==j:
            num_edges = num_edges/2
            network_size = s1*(s1-1)/2
        else:
            network_size = s1*s2
        df.loc[net1, net2]=num_edges
        df_ratio.loc[net1,net2]=num_edges/float(network_size)

In [ ]:
df=df.astype(float)
df_ratio=df_ratio.astype(float)
df

In [ ]:
df_ratio

In [ ]:
mask = np.triu(np.ones_like(df, dtype=bool),k=1)
f, ax = plt.subplots(figsize=(6,6))
hmp=sns.heatmap(df, mask=mask, cmap='Blues',annot=True,linewidths=.5,square=True,linecolor='white',cbar_kws={"shrink": 0.7},annot_kws={"fontsize":12},fmt='g')
ax.set_yticklabels(lst_nets,fontsize=12)
ax.set_xticklabels(lst_nets,fontsize=12)
plt.savefig("{}/canonical_networks/network_neg_canonical_grid_plot_thresh{}.png".format(read_path,thresh), format='png',bbox_inches="tight",dpi=300)

In [ ]:
mask = np.triu(np.ones_like(df_ratio, dtype=bool),k=1)
f, ax = plt.subplots(figsize=(6,6))
hmp=sns.heatmap(df_ratio, mask=mask, cmap='Blues',annot=True,linewidths=.5,square=True,linecolor='white',cbar_kws={"shrink": 0.7},annot_kws={"fontsize":10},fmt='.2f')
ax.set_yticklabels(lst_nets,fontsize=12)
ax.set_xticklabels(lst_nets,fontsize=12)
plt.savefig("{}/canonical_networks/network_neg_canonical_grid_plot_thresh{}_ratio.png".format(read_path,thresh), format='png',bbox_inches="tight",dpi=300)